In [2]:
from sqlalchemy.sql import text
import sqlalchemy as sa
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from datetime import datetime

engine = create_engine('mysql+pymysql://root@localhost:8081/airport-analytics?charset=utf8mb4')

# FOREIGN KEY tailnum_fk(tailnum) REFERENCES planes (tailnum), YES
# FOREIGN KEY dest_fk(dest) REFERENCES airports (faa), YES

In [3]:
airports = pd.read_csv('../static/airports.csv')
airports.fillna(np.nan, inplace = True)
airports.to_sql('airports', con=engine, if_exists='append', index = False)

weather = pd.read_csv('../static/weather.csv')
weather.fillna(np.nan, inplace = True)
weather.replace(r'\s+', np.nan, regex=True, inplace=True)
weather['time_hour'] = pd.to_datetime(weather['time_hour'], errors='coerce').values.astype('datetime64[us]')
weather.to_sql('weather', con=engine, if_exists='append', index = False)

airlines = pd.read_csv('../static/airlines.csv')
airlines.fillna(np.nan, inplace = True)
airlines.to_sql('airlines', con=engine, if_exists='append', index = False)

planes = pd.read_csv('../static/planes.csv')
planes.fillna(np.nan, inplace = True)
planes['speed'].replace(r'\s+', np.nan, regex=True, inplace=True)
planes['year'].replace(r'\s+', np.nan, regex=True, inplace=True)
planes.to_sql('planes', con=engine, if_exists='append', index = False)

In [5]:
flights = pd.read_csv('../static/flights.csv')
flights.fillna(np.nan, inplace = True)
flights['time_hour'] = pd.to_datetime(flights['time_hour'], errors='coerce').values.astype('datetime64[us]')
flights['arr_delay'].replace(r'\s+', np.nan, regex=True, inplace=True)
flights['air_time'].replace(r'\s+', np.nan, regex=True, inplace=True)
flights['arr_time'].replace(r'\s+', np.nan, regex=True, inplace=True)
flights['dep_time'].replace(r'\s+', np.nan, regex=True, inplace=True)
flights['dep_delay'].replace(r'\s+', np.nan, regex=True, inplace=True)

RangeIndex(start=0, stop=336776, step=1)


In [20]:
test = flights[~(flights.dest.isin(airports['faa']))]
test2 = flights[~(flights.tailnum.isin(planes['tailnum']))]
test3 = flights.drop(test, axis=1)

print(len(test2.index))

52606


In [ ]:
for i in range(len(flights.index)):
    try:
        row = flights.iloc[[i]]

        row.to_sql('flights', con=engine, if_exists='append', index = False)

    except Exception as e:
        continue

In [ ]:
for i in range(len(flights.index)):
    try:
        row = flights.iloc[[i]]

        row.to_sql('flights', con=engine, if_exists='append', index = False)

    except Exception as e:
        continue

In [8]:
len(flights.index)

336776

In [6]:
err_array = []
for i, row in flights.head(2).iterrows():
    try:
        newRow = [[row].to_frame()]
        newRow.to_sql('flights', con=engine, if_exists='append', index = False)

    except Exception as e:
        err_array.append(f"{i} : {e}")
        #continue
print(err_array)

["0 : 'list' object has no attribute 'to_frame'", "1 : 'list' object has no attribute 'to_frame'"]


In [ ]:
flights = flights[
    (flights.dest.isin(airports['faa'])) & 
    (flights.tailnum.isin(planes['tailnum'])) &
    (flights.carrier.isin(airlines['carrier'])) &
    (flights.origin.isin(airports['faa'])) &
    (flights.year.isin(weather['year'])) &
    (flights.month.isin(weather['month'])) &
    (flights.day.isin(weather['day'])) &
    (flights.hour.isin(weather['hour'])) &
    (flights.origin.isin(weather['origin']))
    ]


In [ ]:
flights.to_sql('flights', con=engine, if_exists='append', index = False)